In [1]:
import numpy as np
import pandas as pd
import string
from string import digits
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.layers import Input, LSTM, Embedding, Dense, Bidirectional, Concatenate, Dot, Activation, TimeDistributed
from keras.models import Model
from keras.utils import plot_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import seaborn as sns
import matplotlib.pyplot as plt

<h1>Dataset</h1>
<p><a href="https://www.kaggle.com/datasets/dariocioni/c4200m/data"><b>C4 2OOM </b></a> - untuk Grammar Error Correction</p>

In [2]:
new_column_names = ["input", "target"]

n_rows =  100
df1 = pd.read_csv('C:/Users/Azzriel/Desktop/Language Chatbot/data/C4 200M/C4_200M.tsv-00000-of-00010',sep = '\t',nrows = n_rows)
df1.columns = new_column_names
df2 = pd.read_csv('C:/Users/Azzriel/Desktop/Language Chatbot/data/C4 200M/C4_200M.tsv-00001-of-00010',sep = '\t',nrows = n_rows)
df2.columns = new_column_names
df3 = pd.read_csv('C:/Users/Azzriel/Desktop/Language Chatbot/data/C4 200M/C4_200M.tsv-00002-of-00010',sep = '\t',nrows = n_rows)
df3.columns = new_column_names
df4 = pd.read_csv('C:/Users/Azzriel/Desktop/Language Chatbot/data/C4 200M/C4_200M.tsv-00003-of-00010',sep = '\t',nrows = n_rows)
df4.columns = new_column_names
df5 = pd.read_csv('C:/Users/Azzriel/Desktop/Language Chatbot/data/C4 200M/C4_200M.tsv-00004-of-00010',sep = '\t',nrows = n_rows)
df5.columns = new_column_names
df6 = pd.read_csv('C:/Users/Azzriel/Desktop/Language Chatbot/data/C4 200M/C4_200M.tsv-00005-of-00010',sep = '\t',nrows = n_rows)
df6.columns = new_column_names
df7 = pd.read_csv('C:/Users/Azzriel/Desktop/Language Chatbot/data/C4 200M/C4_200M.tsv-00006-of-00010',sep = '\t',nrows = n_rows)
df7.columns = new_column_names
df8 = pd.read_csv('C:/Users/Azzriel/Desktop/Language Chatbot/data/C4 200M/C4_200M.tsv-00007-of-00010',sep = '\t',nrows = n_rows)
df8.columns = new_column_names
df9 = pd.read_csv('C:/Users/Azzriel/Desktop/Language Chatbot/data/C4 200M/C4_200M.tsv-00008-of-00010',sep = '\t',nrows = n_rows)
df9.columns = new_column_names
df10 = pd.read_csv('C:/Users/Azzriel/Desktop/Language Chatbot/data/C4 200M/C4_200M.tsv-00009-of-00010',sep = '\t',nrows = n_rows)
df10.columns = new_column_names
data = pd.concat((df1,df2,df3,df4,df5,df6,df7,df8,df9,df10),axis = 0)

In [3]:
data.head()

,input,target
0,The effect of widespread dud targets two face ...,"1. The effect of ""widespread dud"" targets two ..."
1,tax on sales of stores for non residents are s...,Capital Gains tax on the sale of properties fo...
2,Much many brands and sellers still in the market.,Many brands and sellers still in the market.
3,this is is the latest Maintenance release of S...,This is is the latest maintenance release of S...
4,"Fairy Or Not, I'm the Godmother: no just look,...","Fairy Or Not, I'm the Godmother: Not just a lo..."


In [4]:
data.shape

(1000, 2)

<h2>Data Cleaning</h2>
<h3>Standardisasi data</h3>
<ul>
<li>huruf kata lowercase</li>
<li>simbol koma diubah menjadi COMMA</li>
<li>punktuasi dihilangkan</li>
<li>nomor dihilangkan</li>
</ul>

In [5]:
def clean(lines):
  lines.input=lines.input.apply(lambda x: x.lower())
  lines.target=lines.target.apply(lambda x: x.lower())

  lines.input=lines.input.apply(lambda x: re.sub(",", ' COMMA', x))
  lines.target=lines.target.apply(lambda x: re.sub(",", ' COMMA', x))

  exclude = str.maketrans('', '',string.punctuation)
  lines.input=lines.input.apply(lambda x: x.translate(exclude))
  lines.target=lines.target.apply(lambda x: x.translate(exclude))

  remove_digits = str.maketrans('', '', digits)
  lines.input=lines.input.apply(lambda x: x.translate(remove_digits))
  lines.target=lines.target.apply(lambda x: x.translate(remove_digits))

In [6]:
clean(data)

In [7]:
data.head(5)

,input,target
0,the effect of widespread dud targets two face ...,the effect of widespread dud targets two face...
1,tax on sales of stores for non residents are s...,capital gains tax on the sale of properties fo...
2,much many brands and sellers still in the market,many brands and sellers still in the market
3,this is is the latest maintenance release of s...,this is is the latest maintenance release of s...
4,fairy or not COMMA im the godmother no just lo...,fairy or not COMMA im the godmother not just a...


<h2>Tokenization</h2>

In [8]:
def tokenize_sentence(sentence):
    tokens = tf.strings.split(sentence)
    return tokens

data['input_tokens'] = data['input'].apply(tokenize_sentence)

data['target_tokens'] = data['target'].apply(tokenize_sentence)

print(data)

                                                input  \
0   the effect of widespread dud targets two face ...   
1   tax on sales of stores for non residents are s...   
2    much many brands and sellers still in the market   
3   this is is the latest maintenance release of s...   
4   fairy or not COMMA im the godmother no just lo...   
..                                                ...   
95  removing everything from shimmer to longwear e...   
96  while yankees are high on study prospect gleyb...   
97  our buddies chefs take on a classic rign rob f...   
98  and COMMA the gulf between the “digital haves”...   
99   oh COMMA you know just hangting out in hogsmeade   

                                               target  \
0    the effect of widespread dud targets two face...   
1   capital gains tax on the sale of properties fo...   
2         many brands and sellers still in the market   
3   this is is the latest maintenance release of s...   
4   fairy or not COMMA im the 

<h1>Numerization</h1
<h2>GLoVe embedding <a href="https://nlp.stanford.edu/projects/glove/"> download </a></h2>


In [9]:
embeddings_index = {}
with open('C:/Users/Azzriel/Desktop/Language Chatbot/v4/glove.6B.300d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

def tokens_to_embeddings(tokens):
    embeddings = []
    for token in tokens:
        embedding = embeddings_index.get(token.numpy().decode('utf-8'))
        if embedding is not None:
            embeddings.append(embedding)
    return embeddings

data['input_embeddings'] = data['input_tokens'].apply(tokens_to_embeddings)
data['target_embeddings'] = data['target_tokens'].apply(tokens_to_embeddings)

print(data)


                                                input  \
0   the effect of widespread dud targets two face ...   
1   tax on sales of stores for non residents are s...   
2    much many brands and sellers still in the market   
3   this is is the latest maintenance release of s...   
4   fairy or not COMMA im the godmother no just lo...   
..                                                ...   
95  removing everything from shimmer to longwear e...   
96  while yankees are high on study prospect gleyb...   
97  our buddies chefs take on a classic rign rob f...   
98  and COMMA the gulf between the “digital haves”...   
99   oh COMMA you know just hangting out in hogsmeade   

                                               target  \
0    the effect of widespread dud targets two face...   
1   capital gains tax on the sale of properties fo...   
2         many brands and sellers still in the market   
3   this is is the latest maintenance release of s...   
4   fairy or not COMMA im the 

In [10]:
max_sequence_length = 100  
input_embeddings_padded = pad_sequences(data['input_embeddings'], maxlen=max_sequence_length)
target_embeddings_padded = pad_sequences(data['target_embeddings'], maxlen=max_sequence_length)

x = np.expand_dims(input_embeddings_padded, axis=-1)
y = np.expand_dims(target_embeddings_padded, axis=-1)

padded_data = pd.DataFrame({
    'input_embeddings_padded': list(x),
    'target_embeddings_padded': list(y)
})

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)


In [12]:
input_dim = 300  
latent_dim = 128

encoder_inputs = Input(shape=(max_sequence_length, input_dim))
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(max_sequence_length, input_dim))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(y.shape[-1])  # Output layer
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='mse')  

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 100, 300)          │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_2 (InputLayer)    │ (None, 100, 300)          │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ [(None, 128), (None,      │         219,648 │ input_layer_1[0][0]        │
│                               │ 128), (None, 128)]        │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_1 (LSTM)                 │ [(None, 100, 128), (None, │         219,648 │ input_layer_2[0][0],       │
│                               │ 128), (None, 128)]        │                 │ lstm[0][1], lstm[0][2]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 100, 1)            │             129 │ lstm_1[0][0]               │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 439,425 (1.68 MB)

 Trainable params: 439,425 (1.68 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
model.fit([x_train, y_train[:, :100]], y_train[:, :100], 
          validation_data=([x_val, y_val[:, :100]], y_val[:, :100]), 
          batch_size=64, 
          epochs=10)

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 14s 651ms/step - loss: 0.0050 - val_loss: 0.0036
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 724ms/step - loss: 0.0040 - val_loss: 0.0036
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 910ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 11s 872ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 730ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 13s 903ms/step - loss: 0.0039 - val_loss: 0.0035
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 11s 830ms/step - loss: 0.0037 - val_loss: 0.0035
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 687ms/step - loss: 0.0037 - val_loss: 0.0035
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 724ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 655ms/step - loss: 0.0038 - val_loss: 0.0035


In [14]:
def preprocess_input_string(input_string):
    input_tokens = tokenize_sentence(input_string)
    
    input_embeddings = tokens_to_embeddings(input_tokens)
    
    max_sequence_length = 100  
    padded_input_embeddings = pad_sequences([input_embeddings], maxlen=max_sequence_length)
    
    return padded_input_embeddings

def correct_grammar(input_string, model):
    preprocessed_input = preprocess_input_string(input_string)
    
    corrected_output_embedding = model.predict(preprocessed_input)
    
    corrected_output_tokens = embeddings_to_tokens(corrected_output_embedding)
    
    corrected_output_string = ' '.join(corrected_output_tokens)
    
    return corrected_output_string

input_string = ["This are example sentences.", "This is an example sentence."]
corrected_string = correct_grammar(input_string, model)
print(corrected_string)



AttributeError: 'numpy.ndarray' object has no attribute 'decode'